## Kmodes and SVD

Importing the lib

In [1]:
import pandas as pd
import numpy as np
from km import KMeans
from SVD import Matrix
from kmode import KModes
import matplotlib.pyplot as plt
import math

C:\Users\Srinivasan M\AppData\Local\Temp\ipykernel_9264\55362673.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_data(file_path):
    return pd.read_csv(file_path)


In [3]:
def elbow_method(data, max_clusters=50):
    distortions = []
    for i in range(1, max_clusters + 1):
        km = KModes(i)
        # print(data.shape)
        km.fit(data)
        distortions.append(km.get_cost(data))
    # Plotting the elbow curve
    plt.plot(range(1, max_clusters + 1), distortions, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
    plt.title('Elbow Method')
    plt.show()


Reading the first 10,000 rows of the data.

In [4]:
file_path = "./EncodedCombined2.csv"  # Replace with your dataset file path
data = load_data(file_path)

avg_rating = load_data("./Movie_Rating.csv")
# Assuming the data matrix is constructed with users as rows and movies as columns
data_matrix = data.iloc[:10000,:]  # Exclude first column (user IDs)
# data_matrix.drop(columns=["Zip-code","Title"],inplace=True)
print(data_matrix)


      UserID  MovieID  Rating  Action  Adventure  Animation  Children's  \
0          1     1193       5       0          0          0           0   
1          1      661       3       0          0          1           1   
2          1      914       3       0          0          0           0   
3          1     3408       4       0          0          0           0   
4          1     2355       5       0          0          1           1   
...      ...      ...     ...     ...        ...        ...         ...   
9995     195     2012       3       0          0          0           0   
9996     195     1214       5       1          0          0           0   
9997     195     1215       4       1          1          0           0   
9998     195      541       5       0          0          0           0   
9999     195      542       2       0          0          0           0   

      Comedy  Crime  Documentary  ...  Musical  Mystery  Romance  Sci-Fi  \
0          0      0    

Making necessary data changes.

In [5]:
data_matrix["MovieID"]=data_matrix["MovieID"].astype(np.int16)
data_matrix["UserID"]=data_matrix["UserID"].astype(np.int16)
for col in data_matrix.columns:
    if(col!="MovieID" and col!="UserID"):
        data_matrix[col]=data_matrix[col].astype(np.int8)
print(data_matrix.dtypes)

# print(data_matrix)

UserID         int16
MovieID        int16
Rating          int8
Action          int8
Adventure       int8
Animation       int8
Children's      int8
Comedy          int8
Crime           int8
Documentary     int8
Drama           int8
Fantasy         int8
Film-Noir       int8
Horror          int8
Musical         int8
Mystery         int8
Romance         int8
Sci-Fi          int8
Thriller        int8
War             int8
Western         int8
Gender          int8
Age             int8
Occupation      int8
dtype: object


In [6]:
data_matrix

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Gender,Age,Occupation
0,1,1193,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
1,1,661,3,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,1,10
2,1,914,3,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,1,10
3,1,3408,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
4,1,2355,5,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,195,2012,3,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,25,12
9996,195,1214,5,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,25,12
9997,195,1215,4,1,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,25,12
9998,195,541,5,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,25,12


In [7]:
# print(data_matrix)
data_mat=data_matrix.values
# elbow_method(data_mat)

In [8]:
data_mat = data_mat.astype(float)
data_mat 

array([[1.000e+00, 1.193e+03, 5.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+01],
       [1.000e+00, 6.610e+02, 3.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+01],
       [1.000e+00, 9.140e+02, 3.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+01],
       ...,
       [1.950e+02, 1.215e+03, 4.000e+00, ..., 0.000e+00, 2.500e+01,
        1.200e+01],
       [1.950e+02, 5.410e+02, 5.000e+00, ..., 0.000e+00, 2.500e+01,
        1.200e+01],
       [1.950e+02, 5.420e+02, 2.000e+00, ..., 0.000e+00, 2.500e+01,
        1.200e+01]])

Running the Kmodes

In [9]:
k=45
km = KModes(k)
km.fit(data_mat)
km.cluster_centroids

Cost i.e., sum of delta metrics for all data-points for Iteration 1:  41430
Cost i.e., sum of delta metrics for all data-points for Iteration 2:  38023


[[166,
  2916,
  4,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  18,
  4],
 [178,
  364,
  5,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  25,
  17],
 [192, 21, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 1],
 [44,
  1073,
  5,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  45,
  17],
 [62, 1197, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 35, 3],
 [42, 3699, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 25, 8],
 [24, 590, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 25, 7],
 [166, 296, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 4],
 [123,
  1200,
  3,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  35,
  9],
 [192,
  1923,
  5,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

This function will basically tell the which cluster each of the rows in data_mat will belong to.

In [10]:
clustered_ratings=km.predict(data_mat)
clustered_ratings

[(array([1.000e+00, 1.193e+03, 5.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 1.000e+01]),
  10),
 (array([  1., 661.,   3.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
           1.,  10.]),
  44),
 (array([  1., 914.,   3.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   1.,   0.,   1.,   0.,   0.,   0.,   0.,   1.,
           1.,  10.]),
  4),
 (array([1.000e+00, 3.408e+03, 4.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00, 1.000e+01]),
  6),
 (array([1.000e+00, 2.35

In [11]:
rows_per_clusters=[]
for cluster in range(k):
    rows_per_clusters.append([])

for row in clustered_ratings:
    row_cluster=row[1]
    rows_per_clusters[row_cluster].append(row[0])

rows_per_clusters

[[array([2.000e+00, 2.916e+03, 3.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
         1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 5.600e+01, 1.600e+01]),
  array([2.00e+00, 1.21e+03, 4.00e+00, 1.00e+00, 1.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 1.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 5.60e+01, 1.60e+01]),
  array([2.000e+00, 1.544e+03, 4.000e+00, 1.000e+00, 1.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
         0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
         1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 5.600e+01, 1.600e+01]),
  array([  2., 589.,   4.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.

SVD

In [12]:
user_ids = set(data_matrix['UserID'])
user_ids

{1,
 2,
 15,
 17,
 18,
 19,
 24,
 28,
 33,
 39,
 42,
 44,
 48,
 49,
 58,
 59,
 62,
 88,
 96,
 99,
 102,
 107,
 109,
 120,
 123,
 131,
 136,
 139,
 146,
 150,
 151,
 155,
 166,
 169,
 175,
 178,
 180,
 190,
 192,
 195}

In [13]:
def get_non_zero_eigenvalues(Sigma):
    # Extract diagonal elements of Sigma
    print("Sigma")
    print(Sigma)
    diagonal_elements = np.diag(Sigma)
    print(diagonal_elements)
    # Filter out values not close to zero
    non_zero_eigenvalues = [eigenvalue for eigenvalue in diagonal_elements if not math.isclose(eigenvalue, 0)]

    return non_zero_eigenvalues


Choosing a cluster and finding the movies and ratings in that cluster.

In [14]:
new_user_cluster=2
avgdict = avg_rating.set_index('MovieID')['AvgRating'].to_dict() ## might need to change this
print(avgdict)
avg_dict = {int(key): float(value) for key, value in avgdict.items() if value!='Avg. Rating'}
print(avg_dict)

ratings_A = rows_per_clusters[new_user_cluster]
column_names = [
    "UserID", "MovieID", "Rating", "Action", "Adventure", "Animation", 
    "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", 
    "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", 
    "Thriller", "War", "Western", "Gender", "Age", "Occupation"
]
ratings_df=pd.DataFrame(ratings_A,columns=column_names)
# print(ratings_df)

A=ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')
# print(A)
# A_mat = pd.DataFrame(A)
A_watched=A.copy()
A_mat= A.fillna(avg_dict)
A_mat

{'1': '4.146846413', '2': '3.201141227', '3': '3.016736402', '4': '2.729411765', '5': '3.006756757', '6': '3.878723404', '7': '3.410480349', '8': '3.014705882', '9': '2.656862745', '10': '3.540540541', '11': '3.793804453', '12': '2.3625', '13': '3.262626263', '14': '3.54248366', '15': '2.45890411', '16': '3.793255132', '17': '4.02754491', '18': '3.337579618', '19': '2.480719794', '20': '2.5375', '21': '3.623893805', '22': '3.349206349', '23': '2.857142857', '24': '3.179487179', '25': '3.651020408', '26': '3.53', '27': '2.93442623', '28': '4.055865922', '29': '4.062034739', '30': '3.648648649', '31': '3.113475177', '32': '3.945731304', '33': '3', '34': '3.891490577', '35': '3.314285714', '36': '3.957974138', '37': '3.5', '38': '2.821428571', '39': '3.623348018', '40': '3.933333333', '41': '3.958677686', '42': '2.868778281', '43': '3.445783133', '44': '2.78778135', '45': '3.424632353', '46': '3.108433735', '47': '4.106420405', '48': '2.976439791', '49': '3.740740741', '50': '4.517106001'

MovieID,11.0,14.0,16.0,17.0,21.0,22.0,24.0,25.0,30.0,32.0,...,3742.0,3788.0,3791.0,3810.0,3813.0,3816.0,3886.0,3897.0,3910.0,3949.0
UserID,,,,,,,,,,,,,,,,,,,,,
1.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
2.0,3.793804,3.542484,3.793255,4.027545,1.000000,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
17.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.000000,3.65102,5.000000,5.000000,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
18.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
19.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
28.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
39.0,3.793804,3.542484,3.793255,4.027545,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
44.0,3.793804,3.542484,3.793255,5.000000,3.623894,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132
49.0,3.793804,3.542484,3.793255,4.027545,3.000000,3.349206,3.179487,3.65102,3.648649,3.945731,...,3.970085,3.808943,3.066667,3.13253,3.289157,3.763158,3.0,4.226358,3.82,4.115132


In [15]:
A_mat=A_mat.round(2)
# print(A_mat.dtypes)
A_mat

MovieID,11.0,14.0,16.0,17.0,21.0,22.0,24.0,25.0,30.0,32.0,...,3742.0,3788.0,3791.0,3810.0,3813.0,3816.0,3886.0,3897.0,3910.0,3949.0
UserID,,,,,,,,,,,,,,,,,,,,,
1.0,3.79,3.54,3.79,4.03,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
2.0,3.79,3.54,3.79,4.03,1.00,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
17.0,3.79,3.54,3.79,4.03,3.62,3.35,3.00,3.65,5.00,5.00,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
18.0,3.79,3.54,3.79,4.03,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
19.0,3.79,3.54,3.79,4.03,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
28.0,3.79,3.54,3.79,4.03,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
39.0,3.79,3.54,3.79,4.03,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
44.0,3.79,3.54,3.79,5.00,3.62,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12
49.0,3.79,3.54,3.79,4.03,3.00,3.35,3.18,3.65,3.65,3.95,...,3.97,3.81,3.07,3.13,3.29,3.76,3.0,4.23,3.82,4.12


In [16]:
A = Matrix(A_mat)
U,Sigma,V = A.svd(flg = False)

non_zero_diag = get_non_zero_eigenvalues(Sigma)
U_red,Sigma_red,V_red = A.reduced_svd(int(non_zero_diag[0]))  # Number of components for SVD 

ATA
MovieID    11.0      14.0      16.0      17.0      21.0      22.0      24.0    \
MovieID                                                                         
11.0     273.1897  254.6056  269.1397  291.5392  249.0570  238.5905  233.3630   
14.0     254.6056  241.5688  253.8622  276.3454  238.5620  225.6230  221.1552   
16.0     269.1397  253.8622  267.5538  290.6929  249.6768  237.5370  232.6952   
17.0     291.5392  276.3454  290.6929  317.3362  273.1990  258.3485  253.1864   
21.0     249.0570  238.5620  249.6768  273.1990  244.6704  222.4320  218.7424   
...           ...       ...       ...       ...       ...       ...       ...   
3816.0   273.2764  259.0168  272.4868  296.5476  256.1080  242.1620  237.2888   
3886.0   212.2900  199.6200  210.6600  228.5900  195.7600  186.9000  183.0000   
3897.0   299.6889  282.1014  297.5706  323.0373  276.9216  264.0690  258.6024   
3910.0   274.2270  260.5076  273.4248  297.6536  256.9376  243.0120  238.2140   
3949.0   296.0516  282.1

In [27]:
np.savetxt('U_red.txt', U_red)
np.savetxt('Sigma_red.txt', Sigma_red)
np.savetxt('V_red.txt', V_red)

In [22]:
# new_mat = (U_red.dot(Sigma)).dot(V_red)
new_mat = np.matmul(U_red,np.matmul(Sigma_red,np.transpose(V_red)))
new_mat

array([[3.78999991, 3.53999998, 3.79000003, ..., 4.23000003, 3.81999994,
        4.12000004],
       [3.79000001, 3.54      , 3.79000001, ..., 4.23000001, 3.82000001,
        4.12000001],
       [3.79000003, 3.54      , 3.79      , ..., 4.23      , 3.81999999,
        4.12      ],
       ...,
       [3.79000026, 3.53999994, 3.78999995, ..., 4.22999996, 3.81999987,
        4.11999986],
       [5.00000001, 3.53999999, 3.78999998, ..., 4.22999998, 3.81999997,
        4.11999998],
       [2.00000002, 3.54      , 3.00000001, ..., 3.00000001, 3.82      ,
        4.99999999]])

In [29]:


corrected_array = new_mat
corrected_array

array([[3.78999991, 3.53999998, 3.79000003, ..., 4.23000003, 3.81999994,
        4.12000004],
       [3.79000001, 3.54      , 3.79000001, ..., 4.23000001, 3.82000001,
        4.12000001],
       [3.79000003, 3.54      , 3.79      , ..., 4.23      , 3.81999999,
        4.12      ],
       ...,
       [3.79000026, 3.53999994, 3.78999995, ..., 4.22999996, 3.81999987,
        4.11999986],
       [5.00000001, 3.53999999, 3.78999998, ..., 4.22999998, 3.81999997,
        4.11999998],
       [2.00000002, 3.54      , 3.00000001, ..., 3.00000001, 3.82      ,
        4.99999999]])

Finding the average rating of the movies from the SVD

In [30]:
movie_averages=np.mean(corrected_array,axis=0)
movie_IDs=A_watched.columns
movie_avg_df=pd.DataFrame(movie_averages,index=movie_IDs)
movie_avg_df

,0
MovieID,
11.0,3.759474
14.0,3.564211
16.0,3.748421
17.0,4.081053
21.0,3.522105
...,...
3816.0,3.825263
3886.0,2.947368
3897.0,4.165263


Recommending the Movies for the user.

In [31]:
# Recommend movies for a particular user
def recommend_movies(user_id, users_watched,movie_averages):      
    df=users_watched[users_watched.index==user_id]
    userMovies=[]
    for index, row in df.iterrows():
        for column in df.columns:
            value = row[column]
            # Check if the value is not NaN (not null)
            if not pd.isna(value):
                userMovies.append(column)

    filtered_df = movie_averages[~movie_averages.index.isin(userMovies)]
    filtered_df=filtered_df.reset_index()
    
    movies=pd.read_csv("movies.csv")
    movies=movies.set_index(keys=['MovieID'])
    merged_df=pd.merge(movies,filtered_df,on='MovieID')
    merged_df=merged_df.sort_values(by=0,ascending=False)
    recommendations=merged_df.to_numpy()
    # print(recommendations)
    return recommendations

In [32]:
# Recommend movies for the user
user_id=1
recommended_movies = recommend_movies(user_id,A_watched,movie_avg_df)
print("Recommended movies for User ", user_id)
for movie in recommended_movies[:5]:  # Display top 10 recommended movies
    print(f"Title : {movie[1]} , MovieID : {movie[0]} , Rating : {movie[3]} , Genres : {movie[2]}")  # Assuming the first column is movie titles


Recommended movies for User  1
Title : Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) , MovieID : 2019 , Rating : 4.606315789499392 , Genres : Action|Drama
Title : Godfather, The (1972) , MovieID : 858 , Rating : 4.595789473801125 , Genres : Action|Crime|Drama
Title : Silence of the Lambs, The (1991) , MovieID : 593 , Rating : 4.589473684995558 , Genres : Drama|Thriller
Title : Shawshank Redemption, The (1994) , MovieID : 318 , Rating : 4.586842105157931 , Genres : Drama
Title : Usual Suspects, The (1995) , MovieID : 50 , Rating : 4.570526315819115 , Genres : Crime|Thriller
